# Lineaire regressie in Python
## 1.1 Data laden in Pandas

Voor een recap over lineaire regressie gaan we kijken naar een voorbeeld op de Wine dataset. Een Portgese wijnproducent probeert de kwaliteit van de wijn te controleren door verschillende metingen te doen, zoals de zuurgraad en de overgebleven suiker na gisting. Men wil weten hoe deze factoren samenhangen met de waardering van de wijn, zodat men tijdens het productieproces kan voorspellen hoe goed de wijn wordt en eventueel bijsturen. De dataset bevat metingen op 1599 verschillende rode Vinho Verde wijnen. We focussen in dit experiment op de rode wijn, maar voor liefhebbers is er ook data van witte wijn.

Eerst een stukje code om de dataset te downloaden van het web. Deze code is nu niet belangrijk.

In [1]:
import os
if not os.path.exists("winequality-red.csv"):
    import urllib.request
    f = urllib.request.urlretrieve ("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", \
                                    "winequality-red.csv")

### Inspecteren van de datafile. 

We hebben nu een datafile `winequality-red.csv`. Door de extensie vermoeden we dat de structuur van de file CSV is (Comma Separated Values), waarin de eerste file de kolom headers bevat en elke volgende rij de gemeten data van een wijn. We kunnen de eerste paar rijen bekijken om dat te bevestigen. Omdat in dit geval dit notebook op een Linux server draait, kunnen we een zgn BASH commando gebruiken. Maar je kunt uiteraard ook het bestand bekijken in een text editor.

In [2]:
%%bash
head -n 3 winequality-red.csv

"fixed acidity";"volatile acidity";"citric acid";"residual sugar";"chlorides";"free sulfur dioxide";"total sulfur dioxide";"density";"pH";"sulphates";"alcohol";"quality"
7.4;0.7;0;1.9;0.076;11;34;0.9978;3.51;0.56;9.4;5
7.8;0.88;0;2.6;0.098;25;67;0.9968;3.2;0.68;9.8;5


#### Pandas

Er zijn verschillende libraries die het makkelijk maken om met data te werken. Hier gebruiken we **Pandas**, wat een van de meest gebruikte tools is voor het snel verwerken en analyseren van gestructureerde data. Pandas zet de data in zgn **Dataframes**: tabellen met column headers. De syntax van het werken met Dataframes is conform het werken met andere collecties in Python, maar wijkt wellicht iets af van wat je gewend bent.

#### Laden van de datafile

Als je in Python gebruik wilt maken van een library dan moet je die eenmalig laden met een `import` commando. Bij Pandas is het goed gebruik om de library te hernoemen naar de afkorting `pd` zodat de programmeercode wat compacter en overzichtelijker wordt. We kunnen uit Pandas de `read_csv()` functie gebruiken om een `.csv` bestand in te lezen. Omdat het scheidingsteken tussen kolommen in de datafile een `;` is in plaats van een `,` moeten we `delimeter=';'` meegeven aan de functie. De kolomnamen van het Dataframe worden overgenomen uit header (eerste regel) van de datafile.

In [7]:
import pandas as pd
wijnen = pd.read_csv('winequality-red.csv', delimiter=';')

Nu de data is geladen, kunnen we deze inspecteren. Elke variabele komt in een kolom terecht en elk data sample (in dit geval een Portugese wijn) Pandas heeft daarvoor een aantal handige attributen en functies:

* `dataframenaam.columns`: toont de kolommen
* `dataframenaam`: toont de inhoud (werkt alleen als dit op de laatste regel in een cell staat, het notebook roept dan print(data) aan). Voor kleine tabellen wordt de hele tabel getoond, voor grote tabellen meestal de eerste 30 en de laatste 30 rijen.
* `dataframenaam[rij]`: toont een specifiek rijnummer uit de tabel. Rijen zijn genummerd vanaf 0.
* `dataframenaam[-1]`: toont de laatse rij uit de tabel. Indien rij een negatief getal is wordt vanaf het einde terug geteld.
* `dataframenaam[begin:eind]`: toont de rijen tussen begin (inclusief) en eind (exclusief), en eind mag een negatief getal zijn om vanaf het einde terug te tellen.

NB al deze namen zijn case-sensitive!

In [8]:
wijnen.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [9]:
wijnen[0:5]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In deze case is de target variabele (ook wel afhankelijke of te voorspellen variabele): `quality`

#### Kolommen specificeren

In een Dataframe kunnen we kolommen aanduiden op twee manieren:

- dataframenaam.quality
- dataframenaam['quality']

In [13]:
wijnen.quality[0:5]

0    5
1    5
2    5
3    6
4    5
Name: quality, dtype: int64

Een kleine tip: indien je meer kolommen wilt zien, moet je een verzameling kolommen meegeven. In Python, kun je een lijst kolommen specificeren tussen [].

In [15]:
wijnen[['pH', 'quality']][0:5]

,pH,quality
0,3.51,5
1,3.20,5
2,3.26,5
3,3.16,6
4,3.51,5


Vervolgens kunnen we per kolom kijken wat het domein is met de functie:

- `dataframenaam.columnname.describe()`
- `dataframenaam.columnname.unique()`

We zien dat de waarde voor quality altijd een geheel getal is, dat in het interval [3, ..., 8] ligt en het gemiddelde 5.6 is.

In [17]:
wijnen.quality.unique()

array([5, 6, 7, 4, 8, 3])

In [16]:
wijnen.quality.describe()

count    1599.000000
mean        5.636023
std         0.807569
min         3.000000
25%         5.000000
50%         6.000000
75%         6.000000
max         8.000000
Name: quality, dtype: float64